In [2]:
import os
CODE_DIR = '/home/joshua/Projects/diffae'
os.chdir(f'{CODE_DIR}')

In [3]:
from pytorch_msssim import ms_ssim
import torch
import lpips

l1_criterion = torch.nn.L1Loss(reduction='mean')
l2_criterion = torch.nn.MSELoss(reduction='mean')


def rec_loss(attr_images, generated_images, a):
    ms_ssim_loss = 1 - ms_ssim(attr_images, generated_images, data_range=1, size_average=True)
    l1_loss_value = l1_criterion(attr_images, generated_images)
    return a * ms_ssim_loss + (1 - a) * l1_loss_value


def id_loss(encoded_input_image, encoded_generated_image):
    return l1_criterion(encoded_input_image, encoded_generated_image)


def landmark_loss(input_attr_lnd, output_lnd):
    loss = l2_criterion(input_attr_lnd, output_lnd)
    return loss


def l2_loss(attr_images, generated_images):
    loss = l2_criterion(attr_images, generated_images)
    return loss

In [4]:
from templates import *
from templates_cls import *
from experiment_classifier import ClsModel

import sys
sys.path.append('/home/joshua/Projects/diffae/disentanglement/')

from Configs import Global_Config
from Configs.training_config import config, GENERATOR_IMAGE_SIZE
from Training.trainer import Trainer
from torch.utils.data import DataLoader, random_split
from Models.Encoders.Landmark_Encoder import Landmark_Encoder
from Models.Encoders.ID_Encoder import ID_Encoder
from Models.Encoders.Inception import Inception
from Models.LatentMapper import LatentMapper
from Utils.data_utils import get_w_image, Image_W_Dataset, cycle_images_to_create_diff_order
import time
import torch.utils.data
from tqdm import tqdm
from Losses import id_loss

device = 'cuda:0'
conf = ffhq256_autoenc()
# print(conf.name)
model = LitModel(conf)
state = torch.load(f'checkpoints/{conf.name}/last.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)
model.ema_model.to(device)
# model.ema_model.requires_grad_(True)
model.ema_model.eval()
model.eval()
# torch.set_grad_enabled(True)

BASE_PATH = '/home/joshua/Projects/diffae/disentanglement/'
MOBILE_FACE_NET_WEIGHTS_PATH = BASE_PATH + 'CNN-project-weights/mobilefacenet_model_best.pth.tar'
GENERATOR_WEIGHTS_PATH = BASE_PATH + 'CNN-project-weights/550000.pt'
E_ID_LOSS_PATH = BASE_PATH + 'CNN-project-weights/model_ir_se50.pth'
IMAGE_DATA_DIR = BASE_PATH + 'Dataset/small_image/'
W_DATA_DIR = BASE_PATH + 'Dataset/small_w/'
MODELS_DIR = BASE_PATH + 'Models/'

id_encoder = id_loss.IDLoss(E_ID_LOSS_PATH)
# attr_encoder = torch.load(MODELS_DIR + 'attr_encoder_HPMNYYOTTIXN_1616315604.9508004_0.pt')
attr_encoder = torch.load('/home/joshua/Projects/diffae/checkpoints/attr/attr_ffhq15000_att_mlp.pt')
attr_encoder = Inception(attr_encoder)
# discriminator = Discriminator()
mlp = LatentMapper()
landmark_encoder = Landmark_Encoder.Encoder_Landmarks(MOBILE_FACE_NET_WEIGHTS_PATH)
# mlp = torch.load(MODELS_DIR + 'maper_HPMNYYOTTIXN_1616315604.542634_0.pt')
mlp = torch.load('/home/joshua/Projects/diffae/checkpoints/MLP/mlp_ffhq15000_att_mlp.pt')

id_encoder = id_encoder.to(Global_Config.device)
attr_encoder = attr_encoder.to(Global_Config.device)
# discriminator = discriminator.to(Global_Config.device)
mlp = mlp.to(Global_Config.device)
landmark_encoder = landmark_encoder.to(Global_Config.device)

id_encoder = id_encoder.eval()
lpips_loss = lpips.LPIPS(net='alex').to(Global_Config.device).eval()
landmark_encoder = landmark_encoder.eval()
attr_encoder = attr_encoder.train()

mlp = mlp.train()


Global seed set to 0


Model params: 160.69 M
Loading ResNet ArcFace
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/joshua/anaconda3/envs/proj0/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [7]:
import torch.nn as nn
L2 = nn.MSELoss()

config = {
    'beta1' : 0.5,
    'beta2' : 0.999,
    'lrD' : 0.0004,
    'lrMLP' : 0.00003,
    'lrAttr' : 0.0001,
    'non_adverserial_lr': 6e-5,
    'IdDiffersAttrTrainRatio' : 3, # 1/3
    'batchSize' : 8,
    'R1Param' : 10,
    'lambdaID' : 1,
    'lambdaLND' : 1,
    'lambdaREC' : 1,
    'lambdaL2' : 1,
    'lambdaVGG' : 1,
    'a': 0.84
}

optimizerMLP = torch.optim.Adam(mlp.parameters(), lr=config['lrMLP'], betas=(config['beta1'], config['beta2']))
optimizer_non_adv_M = torch.optim.Adam(list(mlp.parameters()) + list(attr_encoder.parameters()),
                                       lr=config['non_adverserial_lr'], betas=(config['beta1'], config['beta2']))
# image_data = ImageDataset('00000', image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False)
# print(len(image_data))

996


In [6]:
conf = ffhq256_autoenc()
image_data = conf.make_dataset()
# batch = data[10]['img'][None]
for idx, data in enumerate(image_data):
    if idx == 1:
        break
    print(data)

{'img': tensor([[[-0.9686, -1.0000, -0.9843,  ..., -0.9843, -0.9922, -0.9843],
         [-0.9765, -1.0000, -0.9922,  ..., -0.9922, -1.0000, -0.9922],
         [-0.9686, -0.9922, -1.0000,  ..., -0.9922, -1.0000, -1.0000],
         ...,
         [-0.8275, -0.8196, -0.8118,  ...,  0.0196,  0.0275,  0.0353],
         [-0.8745, -0.8588, -0.8510,  ...,  0.0667,  0.0824,  0.0745],
         [-0.8745, -0.8667, -0.8588,  ...,  0.0275,  0.0588,  0.0510]],

        [[ 0.0275, -0.0275,  0.0118,  ..., -0.0667, -0.0980, -0.0902],
         [ 0.0196, -0.0118,  0.0039,  ..., -0.0980, -0.1137, -0.0980],
         [ 0.0275,  0.0039, -0.0039,  ..., -0.1137, -0.1216, -0.1216],
         ...,
         [ 0.2078,  0.2157,  0.2235,  ...,  0.2157,  0.2157,  0.2235],
         [ 0.1529,  0.1686,  0.1765,  ...,  0.2549,  0.2471,  0.2392],
         [ 0.1529,  0.1608,  0.1686,  ...,  0.2157,  0.2235,  0.2157]],

        [[ 0.1451,  0.0902,  0.1451,  ...,  0.2078,  0.1843,  0.1922],
         [ 0.1373,  0.1059,  0.1373, 

In [8]:
from torch.autograd import Variable
# def toggle_grad(model, requires_grad):
#     for p in model.parameters():
#         p.requires_grad_(requires_grad)


# toggle_grad(model, True)

epochs = 1
for epoch in range(epochs):
    for idx, data in enumerate(image_data):
        batch = data['img'][None]
        torch.cuda.empty_cache()
        gen_img = batch.detach().clone()
        id_images = batch.cuda()
        generated_img = Variable(gen_img, requires_grad = True)
        generated_img = generated_img.cuda()
#         print(generated_img)
        attr_batch = batch*0.5+0.5
        attr_images = attr_batch.cuda()
#         xT_true = xT_list[idx][None]
#         xT_true = xT_true.cuda()
        
        total_loss = torch.tensor(0, dtype=torch.float, device=Global_Config.device)
       
        id_vec = id_encoder.extract_feats(id_images)
        attr_vec = torch.squeeze(attr_encoder(attr_images))[None]
        encoded_vec = torch.cat((id_vec,attr_vec), dim=1)
        with torch.no_grad():
            semantic_vec_true = model.encode(id_images)
            xT_true = model.encode_stochastic(id_images, semantic_vec_true, T=20)
        semantic_vec = mlp(encoded_vec)

        generated_img = model.render(xT_true, semantic_vec, T=20)

#         id_encoder.zero_grad()
#         landmark_encoder.zero_grad()
        
        ## -1 to 1
#         id_loss_val = config['lambdaID'] * id_encoder(generated_img*2-1, id_images)
#         total_loss += id_loss_val
        
        ## 0 to 1
#         rec_loss_val = config['lambdaREC'] * rec_loss(attr_images, generated_img, config['a'])
#         total_loss += rec_loss_val
        
        l2_loss_val = config['lambdaL2'] * l2_loss(attr_images, generated_img)
        total_loss += l2_loss_val
        
        ## -1 to 1
#         vgg_loss_val = torch.mean(config['lambdaVGG'] * lpips_loss(generated_img*2-1, id_images))
#         total_loss += vgg_loss_val
        
        optimizer_non_adv_M.zero_grad()
        total_loss.backward()
        optimizer_non_adv_M.step()
        
        if idx%20==0:
            print('ID:', idx, 'l2_loss_val:',total_loss.item())
#         print('id_loss_val:',id_loss_val.item(),'rec_loss_val:',rec_loss_val.item(),'l2_loss_val:',l2_loss_val.item(),'vgg_loss_val:',vgg_loss_val.item())
#         print(f"\n loss:", total_loss.item(), 'idx:', idx)

#     MLP_losses.append(g_error)
#     D_losses.append((error_real + error_fake) /2)

#     if idx % 5 == 0:
#         with torch.no_grad():
#             plot_w_image(mlp(test_vec)[0])

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 9.78 GiB total capacity; 7.59 GiB already allocated; 13.19 MiB free; 7.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
batch

tensor([[[[-0.9922, -0.9922, -0.9922,  ..., -0.0902, -0.2471, -0.2314],
          [-0.9922, -0.9922, -0.9922,  ..., -0.2157, -0.2392, -0.3333],
          [-0.9922, -0.9922, -0.9922,  ..., -0.2314, -0.2078, -0.3098],
          ...,
          [ 0.5765,  0.4745,  0.4588,  ...,  0.7098,  0.7020,  0.6941],
          [ 0.6157,  0.6392,  0.5843,  ...,  0.7176,  0.7098,  0.7020],
          [ 0.6314,  0.5843,  0.4667,  ...,  0.7412,  0.7176,  0.7098]],

         [[-0.9922, -0.9922, -0.9922,  ..., -0.2549, -0.3961, -0.4118],
          [-0.9922, -0.9922, -0.9922,  ..., -0.4510, -0.3725, -0.4902],
          [-0.9922, -0.9922, -0.9922,  ..., -0.4902, -0.3490, -0.4745],
          ...,
          [ 0.5451,  0.4118,  0.3961,  ..., -0.4745, -0.4902, -0.4980],
          [ 0.6157,  0.6314,  0.5765,  ..., -0.4667, -0.4588, -0.4824],
          [ 0.6078,  0.5608,  0.4588,  ..., -0.4588, -0.4431, -0.4667]],

         [[-1.0000, -1.0000, -0.9922,  ..., -0.2471, -0.3882, -0.3725],
          [-0.9922, -0.9922, -